In [ ]:
import pandas as pd
import numpy as np

# import sklearn
# print(sklearn.__all__)
X_tra = pd.read_csv('data/X_train.csv')  #문제지    1
y_tra = pd.read_csv('data/y_train.csv')  #문제지답안  1
test = pd.read_csv('data/X_test.csv')   #시험지
train = pd.merge(X_tra, y_tra, on='cust_id')

# print(test.info())
#,주구매상품   주구매지점
sub_df = pd.DataFrame()
sub_df["cust_id"] = test["cust_id"]
print(sub_df.head())
train = train.drop(['cust_id'], axis=1)
test = test.drop(['cust_id'], axis=1)
train = train.fillna(0)
test = test.fillna(0)

# print(train.isnull().sum())
# print(test.isnull().sum())

df = pd.concat([train, test])
df=pd.get_dummies(df)


train = df.iloc[:3500]
test = df.iloc[3500:]
test = test.drop("gender", axis=1)

print(train.shape)
print(test.shape)

missing_col=print(set(train) - set(test))
print(missing_col)


# import sklearn.preprocessing
# print(dir(sklearn.preprocessing))
y_df= train.gender
X_df = train.drop('gender', axis=1)

from sklearn.preprocessing import RobustScaler, StandardScaler
ss = StandardScaler()
ss.fit(X_df)
train_sc = ss.transform(X_df)
test_sc = ss.transform(test)


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_sc, y_df, test_size=0.2, random_state=0, stratify = y_df)

# from sklearn.ensemble import 
# print(dir(sklearn.ensemble))
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score


rf = RandomForestClassifier(max_depth = 6).fit(X_train, y_train)
rf_prob = rf.predict_proba(X_test)[:,1]
print("rf", roc_auc_score(y_test, rf_prob))


lr = LogisticRegression(random_state = 0).fit(X_train, y_train)
lr_prob = lr.predict_proba(X_test)[:,1]   #7:3  3가지고 내부적 점수 --> 검증
print("lr",roc_auc_score(y_test, lr_prob))


sub_answer = lr.predict_proba(test_sc)  #2482문제 자.. 진짜 풀어봐...
# sub_df["cust_id"] = test["cust_id"]
sub_df["gender"] = sub_answer[:,1].reshape(-1,1)
print(sub_df.head())


sub_df.to_csv("002007678.csv", index=False)
submit = pd.read_csv("002007678.csv")
print(submit.head())


# trian  fit transform 1/10 다해라
# test는 transform만. fit 다시하면 안됨.